In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")
sys.path.append(".")
import copy
import torch
import pickle
import numpy as np
from datetime import datetime
import plotly
plotly.offline.init_notebook_mode(connected=True)

import robust_value_approx.train_value as train_value
import robust_value_approx.utils as utils
import plotting_utils

# Double Integrator Example

In [ ]:
import double_integrator_utils

vf = double_integrator_utils.get_value_function(N=5)

x0_lo = -1 * torch.ones(vf.sys.x_dim, dtype=vf.dtype)
x0_up = 1 * torch.ones(vf.sys.x_dim, dtype=vf.dtype)

# validation options
num_breaks_validation = [100] * vf.sys.x_dim

# file options
sys_name = 'double_int'
x_samples_file = '../data/learn_value_function_' + sys_name + '_x'
v_samples_file = '../data/learn_value_function_' + sys_name + '_v'
model_file = '../data/' + sys_name

# neural network options
nn_width = 16
nn_depth = 1

# setting up adversarial training options
train_opt = train_value.AdversarialWithBaselineTrainingOptions()
train_opt.num_iter_desired = 1000
train_opt.num_steps_between_sampling = 200
train_opt.init_buffer_size = 10
train_opt.num_rand_extra = 0
train_opt.num_x_adv_opt = 5
train_opt.x_adv_max_iter = 2
train_opt.x_adv_conv_tol = 1e-5
train_opt.x_adv_lr = .2
train_opt.batch_size = 30
train_opt.max_buffer_size = 100000

num_training_run = 5

# Vertical Ball Paddle Example

In [ ]:
import ball_paddle_utils

vf = ball_paddle_utils.get_value_function_vertical(N=5)

x0_lo = torch.Tensor([1.5, .15, -5., -1.]).type(vf.dtype)
x0_up = torch.Tensor([2., .15, 1., 5.]).type(vf.dtype)

# validation options
num_breaks_validation = [20, 1, 20, 20]

# data file options
sys_name = 'ball_paddle_vertical'
x_samples_file = '../data/learn_value_function_' + sys_name + '_x'
v_samples_file = '../data/learn_value_function_' + sys_name + '_v'
model_file = '../data/' + sys_name

# neural network options
nn_width = 32
nn_depth = 1

# setting up adversarial training options
train_opt = train_value.AdversarialWithBaselineTrainingOptions()
train_opt.num_iter_desired = 1000
train_opt.num_steps_between_sampling = 200
train_opt.init_buffer_size = 60
train_opt.num_rand_extra = 0
train_opt.num_x_adv_opt = 10
train_opt.x_adv_max_iter = 3
train_opt.x_adv_conv_tol = 1e-4
train_opt.x_adv_lr = .25
train_opt.batch_size = 60
train_opt.max_buffer_size = 100000

num_training_run = 2

In [ ]:
# checking the spread of trajectories for sanity check
fig = plotting_utils.rollout_range(vf, x0_lo, x0_up, [0, 1], ["ball", "paddle"], n=50)
fig.show()

# SLIP Walking Model Example

In [ ]:
import slip_utils

vf, slip = slip_utils.get_value_function(torch.Tensor([6., 1.25, 0.]), N=3)

In [ ]:
x0_lo = torch.Tensor([0, .95, 4.]).type(vf.dtype)
x0_up = torch.Tensor([0, 1.25, 9.]).type(vf.dtype)

# validation options
num_breaks_validation = [1, 100, 100]

# data file options
sys_name = 'slip'
x_samples_file = '../data/learn_value_function_' + sys_name + '_x'
v_samples_file = '../data/learn_value_function_' + sys_name + '_v'
model_file = '../data/' + sys_name

# neural network options
nn_width = 64
nn_depth = 1

# setting up adversarial training options
train_opt = train_value.AdversarialWithBaselineTrainingOptions()
train_opt.num_iter_desired = 1000
train_opt.num_steps_between_sampling = 200
train_opt.init_buffer_size = 100
train_opt.init_num_train_steps = 1000
train_opt.num_rand_extra = 0
train_opt.num_x_adv_opt = 5
train_opt.x_adv_max_iter = 3
train_opt.x_adv_conv_tol = 1e-4
train_opt.x_adv_lr = .1
train_opt.batch_size = 60
train_opt.max_buffer_size = 100000

num_training_run = 1

In [ ]:
# checking the spread of trajectories for sanity check
fig = plotting_utils.rollout_range(vf, x0_lo, x0_up, [0, 1], ["x", "y"], n=100)
fig.show()

# Generating Validation Data

In [ ]:
x_samples_validation, v_samples_validation = vf.get_value_sample_grid(x0_lo, x0_up, num_breaks_validation, update_progress=True)

In [ ]:
torch.save(x_samples_validation, x_samples_file + '_validation.pt')
torch.save(v_samples_validation, v_samples_file + '_validation.pt')

In [ ]:
import plotly.graph_objs as go
fig = go.Figure()
fig.add_trace(go.Scatter(
    y = v_samples_validation.squeeze(),
))
fig.show()

# Or Loading It From File

In [ ]:
x_samples_validation = torch.load(x_samples_file + '_validation.pt')
v_samples_validation = torch.load(v_samples_file + '_validation.pt')

# Adversarial Training

In [ ]:
adv = train_value.AdversarialWithBaseline(vf, x0_lo, x0_up,
                                          nn_width=nn_width, nn_depth=nn_depth,
                                          x_samples_validation=x_samples_validation,
                                          v_samples_validation=v_samples_validation)
state_log = []

In [ ]:
for _ in range(num_training_run):
    adv.train(train_opt)
    state_log.append(adv.get_state())

In [ ]:
# 2D plots of sample buffers
fig = plotting_utils.buffer_plot(state_log, x0_lo, x0_up, 1, 2)
fig.show()

In [ ]:
# 3D plots of adversarial samples (only integrator for now)
fig = plotting_utils.bilevel_plot(state_log[2], adv, 0, 1, show_buffer=False)
fig.show()

In [ ]:
# plot training losses
fig = plotting_utils.training_loss(state_log[0], window=100)
fig.show()

In [ ]:
training_runs = []

In [ ]:
# multiple run and plot validation validation delta
num_runs = 5
for run_i in range(num_runs):
    adv = train_value.AdversarialWithBaseline(vf, x0_lo, x0_up,
                                              nn_width=nn_width, nn_depth=nn_depth,
                                              x_samples_validation=x_samples_validation,
                                              v_samples_validation=v_samples_validation)
    state_log = []
    for _ in range(num_training_run):
        adv.train(train_opt)
        state_log.append(adv.get_state())
    training_runs.append(state_log)
    utils.update_progress((run_i + 1) / num_runs)

In [ ]:
# improvement on test set
fig = plotting_utils.validation_delta([s_log[-1] for s_log in training_runs])
fig.show()

In [ ]:
# save the results if needed
now = datetime.now()
pickle.dump(training_runs, open("training_runs_" + sys_name + "_" + now.strftime("%m%d%Y%H%M%S") + ".p", "wb"))

In [ ]:
# or load an old result
training_runs = pickle.load(open("training_runs_double_int_01252020175702.p", "rb"))

# Saving Models

In [ ]:
torch.save(adv.baseline_model, model_file + '_baseline_model.pt')
torch.save(adv.robust_model, model_file + '_robust_model.pt')